### Learning Representations by Propagating Errors (Backprop)

# Convincing myself of the delta rule for gradient descent
$\delta_{p}w_{ji} = \eta(t_{pj}-o_{pj})i_{pi}$

Where p is the pattern or the combination of input x's and output y,

$\eta$ is the learning rate

$t_{pj}$ is the target of the j'th output for pattern p

$o_{pj}$ is our prediction of the j'th output for pattern p

$i_{pi}$ is the value of the ith element in the input for pattern p

Imagine we're trying to learn AND

|x1 | x2 | y |
|-|-|-|
|0|0|0|
|0|1|0|
|1|1|1|

This is linearly seperable, so the network should be able to learn it. Let's initialize a simple network with one node and work through it by hand. It's a bit tedious to do this in latex / charts so I'll just put this image here.

The delta rule does work and makes sense to me.

In [57]:
# Now let's try to write some code from scratch that allows us to construct these types of networks.
# Note: this code isn't very good or safe, it's just to understand how it works / learns
import random

# network = Network(fan_in, fan_out)
# network.forward(x)
# network.update(x, y, learning_rate)

class Network:
    # need to store fan_in x fan_out connections
    connections = {}
    fan_in = 0
    fan_out = 0
    
    def __init__(self, fan_in, fan_out):
        for x in range(fan_in):
            for y in range(fan_out):
                self.connections[(x,y)] = random.random()
        self.fan_in = fan_in
        self.fan_out = fan_out

    def threshold(self, val):
        if val <= 0.5:
            return 0
        return 1
                
    def forward(self, input):
        out = [0 for i in range(self.fan_out)]
        for y in range(self.fan_out):
            for x in range(self.fan_in):
                out[y] += input[x] * self.connections.get((x, y))
        out = [self.threshold(val) for val in out]
        return out

    def update(self, input, target, learning_rate=0.01):
        # this is where we apply the generalized delta rule for each connection
        forward_pass_vals = self.forward(input)

        for y in range(self.fan_out):
            for x in range(self.fan_in):
                difference = target[y] - forward_pass_vals[y]
                delta = learning_rate * difference * input[x]
                # print(difference)
                self.connections[(x,y)] = self.connections.get((x,y)) + delta

In [71]:
from IPython.display import display, HTML

# Now let's test if we can learn AND
network = Network(2, 1)
# "training data"
data = [([0,0],0), ([0,1],0), ([1,0],0), ([1,1],1)]

# Before training
print("Before Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')

# now let's "train" 100 times
for i in range(100):
    for example in data:
        network.update(example[0], [example[1]])

# After training
print("\nAfter Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')


Before Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 0. Got: 1.00 --> WRONG
Input [1, 0], should be 0. Got: 0.00 --> CORRECT
Input [1, 1], should be 1. Got: 1.00 --> CORRECT

After Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 0. Got: 0.00 --> CORRECT
Input [1, 0], should be 0. Got: 0.00 --> CORRECT
Input [1, 1], should be 1. Got: 1.00 --> CORRECT


In [72]:
# Now let's test if we can learn OR
network = Network(2, 1)
# "training data"
data = [([0,0],0), ([0,1],1), ([1,0],1), ([1,1],1)]

# Before training
print("Before Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')

# now let's "train" 100 times
for i in range(100):
    for example in data:
        network.update(example[0], [example[1]])

# After training
print("\nAfter Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')


Before Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 1. Got: 1.00 --> CORRECT
Input [1, 0], should be 1. Got: 0.00 --> WRONG
Input [1, 1], should be 1. Got: 1.00 --> CORRECT

After Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 1. Got: 1.00 --> CORRECT
Input [1, 0], should be 1. Got: 1.00 --> CORRECT
Input [1, 1], should be 1. Got: 1.00 --> CORRECT


Cool so we can learn AND and OR, but the paper says, and it makes sense that we cannot learn XOR as it is not linearly seperable. So let's see what happens if we try to do that.

In [85]:
# Now let's test if we can learn XOR
network = Network(2, 1)
data = [([0,0],0), ([0,1],1), ([1,0],1), ([1,1],0)]

# Before training
print("Before Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')

for i in range(100):
    for example in data:
        network.update(example[0], [example[1]])
# After training
print("\nAfter Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')


Before Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 1. Got: 0.00 --> WRONG
Input [1, 0], should be 1. Got: 1.00 --> CORRECT
Input [1, 1], should be 0. Got: 1.00 --> WRONG

After Training
Input [0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1], should be 1. Got: 0.00 --> WRONG
Input [1, 0], should be 1. Got: 0.00 --> WRONG
Input [1, 1], should be 0. Got: 1.00 --> WRONG


No matter how many times I run it, it will not learn. In the paper it mentions that with a single extra bit the network is able to learn.

In [101]:
# Now let's test if we can learn XOR with an extra bit
network = Network(3, 1)
data = [([0,0,0],0), ([0,1,0],1), ([1,0,0],1), ([1,1,1],0)]

# Before training
print("Before Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')

# now let's "train" 100 times
for i in range(1000):
    for example in data:
        network.update(example[0], [example[1]])

# After training
print("\nAfter Training")
for example in data:
    output = network.forward(example[0])[0]
    correct = abs(output - example[1]) < 0.5
    status = f"\033[92mCORRECT\033[0m" if correct else f"\033[91mWRONG\033[0m"
    print(f'Input {example[0]}, should be {example[1]}. Got: {output:.2f} --> {status}')


Before Training
Input [0, 0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1, 0], should be 1. Got: 0.00 --> WRONG
Input [1, 0, 0], should be 1. Got: 1.00 --> CORRECT
Input [1, 1, 1], should be 0. Got: 1.00 --> WRONG

After Training
Input [0, 0, 0], should be 0. Got: 0.00 --> CORRECT
Input [0, 1, 0], should be 1. Got: 1.00 --> CORRECT
Input [1, 0, 0], should be 1. Got: 1.00 --> CORRECT
Input [1, 1, 1], should be 0. Got: 0.00 --> CORRECT


Interesting, as the paper confirms it now "learns" the pattern. 

## Now let's see the derivation that proves that this rule is a gradient descent on the error

If we specify the error metric for a single example as

$\frac{1}{2}\sum(t_{pj}-o_{pj})^2$

Then we want to show that the derivative of this is proportional to to the change specified in the rule above.

To do this we can just do a bit of calculus with the chain rule.

If we, for simplicities sake just imagine there's only one output and one input i

$E_{p} = \frac{1}{2}\sum(y - (w * i))^2$

Then if we apply the chain rule we get the derivative as

$-i(y-(w*i))$

Which if we're trying to minimize, we multiply by -1, giving us the term from rule from above!

$i(y-\hat y)$

Now if we take a "step" in that direction, as given by the learning rate then we will minimize the error.

However we did simplify here and assume there's only one input i and one output j for the math.

Now what we can do however is observe that because the error metric is the summation of the errors on all the examples, we know the calculus sum rule means you can just add the derivatives together. So sum up the derivatives for the error with respect to each of the weights for all patterns and that gives us our overall gradient for the weight across the entire set. The interesting thing is that the authors note that if you update the weights after each pattern it's not a true gradient descent on E. I believe however the modern way is to compute the gradients for the entire input batch and sum the gradients together meaning it is a true descent on E.